In [ ]:
using QPControl
using RigidBodyDynamics
using RigidBodyDynamics.Contact
using MeshCatMechanisms
using RigidBodySim
using StaticArrays
using BenchmarkTools
using Compat
using Compat.Test
using SimpleQP

In [ ]:
world = RigidBody{Float64}("world")
mechanism = Mechanism(world; gravity=SVector(0, 0, -9.81))

frame = CartesianFrame3D("core")
inertia = SpatialInertia(frame, SDiagonal(1., 1, 1), SVector(0., 0, 0), 10.)
core = RigidBody(inertia)
joint = Joint("floating_base", Prismatic(SVector(0., 0, 1)))
attach!(mechanism, world, core, joint)
position_bounds(joint) .= RigidBodyDynamics.Bounds(0, 10)
velocity_bounds(joint) .= RigidBodyDynamics.Bounds(-1000, 1000)
effort_bounds(joint) .= RigidBodyDynamics.Bounds(0, 0)

frame = CartesianFrame3D("foot")
inertia = SpatialInertia(frame, SDiagonal(0.05, 0.05, 0.05), SVector(0., 0, 0), 1.0)
foot = RigidBody(inertia)
joint = Joint("foot_extension", Prismatic(SVector(0., 0, 1)))
attach!(mechanism, core, foot, joint)
position_bounds(joint) .= RigidBodyDynamics.Bounds(-1., -0.5)
velocity_bounds(joint) .= RigidBodyDynamics.Bounds(-1000, 1000)
effort_bounds(joint) .= RigidBodyDynamics.Bounds(-200, 200)


floor = HalfSpace3D(Point3D(default_frame(world), 0., 0, 0), FreeVector3D(default_frame(world), 0., 0, 1))
add_environment_primitive!(mechanism, floor)
contactmodel = SoftContactModel(hunt_crossley_hertz(k = 500e3), ViscoelasticCoulombModel(0.8, 20e3, 100.))
add_contact_point!(foot, Contact.ContactPoint(Point3D(default_frame(foot), 0., 0, 0), contactmodel))

In [ ]:
function initialize!(state::MechanismState)
    set_configuration!(state, [1, -1])
    set_velocity!(state, [0, 0])
    set_additional_state!(state, zeros(Float64, num_additional_states(state)))
    state
end

In [ ]:
mvis = MechanismVisualizer(mechanism, Skeleton(randomize_colors=true))
open(mvis)
copy!(mvis, initialize!(MechanismState(mechanism)))

In [ ]:
# create optimizer
using MathOptInterface
using OSQP.MathOptInterfaceOSQP
const MOI = MathOptInterface
using Gurobi

function defaultoptimizer()
    GurobiOptimizer(OutputFlag=0, TimeLimit=1)
end

In [ ]:
mpc = QPControl.MPCController{QPControl.ContactPoint{4}}(mechanism, defaultoptimizer())

stages = QPControl.addstages!(mpc, 1, 0.001)

Q = Diagonal([10, 10, 0.01, 0.01])
xf = vcat(stages[end].q, stages[end].v) - [1, -1, 0, 0]
R = Diagonal([1e-8, 1e-8])
objective = @expression xf' * Q * xf
for stage in stages
    objective = @expression objective + stage.v̇' * R * stage.v̇

    for body in bodies(mechanism)
        for point in RigidBodyDynamics.contact_points(body)
            position = location(point)
            μ = point.model.friction.μ
            contact = addcontact!(mpc, stage, position, floor, μ)
            contact.maxnormalforce[] = 1e6 # TODO
        end
    end
end

@objective mpc.qpmodel Minimize objective

In [ ]:
# simulate
state = initialize!(MechanismState(mechanism))
set_configuration!(state, [3.0, -1.0])
Δt = 1 / 500
pcontroller = PeriodicController(similar(velocity(state)), Δt, mpc)
# TODO: add damping
dynamics = Dynamics(mechanism, pcontroller)
problem = ODEProblem(dynamics, state, (0., 10.))

In [ ]:
τ = similar(velocity(state))
@benchmark $mpc($τ, 0.0, $state)

In [ ]:
sol = solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6)
@time sol = solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6)
@test sol.retcode == :Success

In [ ]:
#NBSKIP
RigidBodySim.animate(mvis, state, sol)